In [139]:
import struct
import numpy as np

In [140]:
def read_from_origin():
    with open("origin_compression.txt", 'r') as f:
        source = f.read()
    return source

In [141]:
def split_data(data):
    folds = []
    ys = len(data) % 8
    if ys != 0:
        data += "0" * ys
        print("ys:",ys)
    size = len(data)
    num_folds = int(size / 8)
    print("size :", size)
    print("num_folds:", num_folds)
    i = 0
    index_split = np.array_split(np.arange(size),indices_or_sections=num_folds)
    for i in index_split:
        cell = data[i[0]:i[-1] + 1]
        folds.append(cell)
    return folds

In [142]:
def refine_list(data):
    lists = []
    for i in data:
        if i not in lists:
            lists.append(i)
    return sorted(lists)

In [143]:
def LWZ(list_bytes, dicts):
    p = ""
    result = []
    last = 256
    for c in list_bytes:
        pc = p + c
        if pc in dicts:
            p = pc
        else:
            result.append(dicts[p])
            dicts[pc] = last
            last += 1
            p = c
    return result, dicts

In [144]:
def convert_to_byte(result):
    b_result = []
    i = 0
    code = ""
    while i < len(result):
        b_result.append(bin(result[i])[2:])
        temp = bin(result[i])[2:]
        code += temp
        i = i + 1
    return code

In [145]:
source = read_from_origin()
print("原始数据的长度：",len(source))
print(type(source))

原始数据的长度： 271112
<class 'str'>


In [146]:
sdata = split_data(source)

size : 271112
num_folds: 33889


In [147]:
print(len(sdata[0]))

8


In [148]:
dicts = dict(zip(refine_list(sdata), [i for i in range(len(refine_list(sdata)))]))

In [149]:
result, dicts = LWZ(sdata, dicts)

In [150]:
code = convert_to_byte(result)

In [151]:
scode = split_data(code)

ys: 4
size : 226944
num_folds: 28368


In [152]:
size = len(scode)
with open('result.txt', 'wb+') as f:
    i = 0
    while i < size:
        wb = struct.pack('B', int(scode[i],2))
        f.write(wb)
        i = i + 1